In [1]:
#!pip install --upgrade google-api-python-client
#!pip install --upgrade google-auth-oauthlib google-auth-httplib2
#!pip install pandas
#!pip install os
import pandas as pd
import os
import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
CLIENT_SECRETS_FILE = "client_secret.json"
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
def get_authenticated_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    credentials = flow.run_console()
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)
# Remove keyword arguments that are not set
def remove_empty_kwargs(**kwargs):
    good_kwargs = {}
    if kwargs is not None:
        for key, value in kwargs.items():
            if value:
                good_kwargs[key] = value
    return good_kwargs

In [233]:
client = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=787554103224-ifu28o6san95n62u5dsu3u4hut0nf2qe.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=OYSCdQ29hBLINnYoFqQcvBjRuILqJK&prompt=consent&access_type=offline
Enter the authorization code: 4/1AY0e-g7zmKjH6xLgCrNd7rSS3ew77B2e1fok_MONFmCK07WrTUYV5YNI-70


In [2]:
path = os.getcwd()
path_bases = os.getcwd() + '\\bases_coletadas\\'

In [11]:
def youtube_keyword(client, **kwargs):    
    kwargs = remove_empty_kwargs(**kwargs)
    response = client.search().list(
        **kwargs
        ).execute()    
    return response

In [3]:
def youtube_search (criteria='',max_res=1, token = None):   
    #create lists and empty dataframe
    titles = []
    videoIds = []
    channelIds = []
    resp_df = pd.DataFrame()
    
    while len(titles) < max_res:
        response = youtube_keyword(client,
                        part='id,snippet',
                        maxResults=50,
                        q=criteria,
                        videoCaption='closedCaption',
                        type='video', 
                        videoDuration='long',
                        pageToken=token) 
                                         
        for item in response['items']:        
            titles.append(item['snippet']['title'])
            channelIds.append(item['snippet']['channelTitle'])
            videoIds.append(item['id']['videoId'])
        
        try:
            token = response['nextPageToken']
        except:
            break
        
    resp_df['title'] = titles
    resp_df['channelId'] = channelIds
    resp_df['videoId'] = videoIds
    resp_df['subject'] = criteria
    
    return resp_df

def get_all_ccs(videos, lang='en'):
    base_url = 'https://www.youtube.com/watch?v='
    for vid in videos:
        url = base_url + vid
        cmd = ["youtube-dl","--skip-download","--write-sub",
               "--sub-lang",lang,url]
        os.system(" ".join(cmd))

In [4]:
#!pip install webvtt-py
import webvtt
def convert_vtt(filenames):    
    #create an assets folder if one does not yet exist
    if os.path.isdir('{}/assets'.format(os.getcwd())) == False:
        os.makedirs('assets')
    #extract the text and times from the vtt file
    for file in filenames:
        captions = webvtt.read(file)
        text_time = pd.DataFrame()
        text_time['text'] = [caption.text for caption in captions]
        text_time['start'] = [caption.start for caption in captions]
        text_time['stop'] = [caption.end for caption in captions]
        text_time.to_csv('assets/{}.csv'.format(file[:-4]),index=False) #-4 to remove '.vtt'
        #remove files from local drive
        os.remove(file)

In [281]:
ids_novo = youtube_search(criteria='random', max_res=2000)
ids = pd.read_excel(path_bases + 'ids.xlsx')
ids_final = pd.concat([ids, ids_novo], axis=0)
ids_final.to_excel(path_bases + 'ids.xlsx')
print("Funcionou! Id's extraidos")

Funcionou! Id's extraidos


In [282]:
ids_final.shape[0]

16066

In [253]:
ids = pd.read_excel(path_bases + 'ids.xlsx')
get_all_ccs(ids.videoId)

In [255]:
subtitles = [file for file in os.listdir() if file[-3:] == 'vtt']
convert_vtt(subtitles)

In [256]:
csv_files = [os.fsdecode(file) for file in os.listdir(os.getcwd()+'/assets') if os.fsdecode(file).endswith('.csv')]
path = 'assets/'
for filename in csv_files:
    os.rename(os.path.join(path, filename), os.path.join(path, filename.replace(' ', '')))
#to verify the results
    


In [258]:
clean_csv = [os.fsdecode(file) for file in os.listdir(os.getcwd()+'/assets')]

vidText = []
csv_vidid = []

for file in clean_csv:
    df = pd.read_csv(path+file)
    df.text = df.text.apply(lambda x: str(x))
    text = " ".join(df.text)
    vidText.append(text)
    csv_vidid.append(file[-18:-7])

In [283]:
videos_df = pd.DataFrame()
videos_df['video_title'] = clean_csv
videos_df['video_captions'] = vidText
videos_df['video_id'] = csv_vidid

videos_df

,video_title,video_captions,video_id
0,#007_NewCityProductionsPodcast-CurtisMcCoshhow...,[Becky] Hi everybody welcome back to another e...,RX--L0xnnDo
1,#1-JoshGroban—TheLinkBetweenMusicandMentalHeal...,"(upbeat music) - [Marjorie] Hi, this\nis Marjo...",kNxmPdHVtp4
2,#16ទៅប្រមូលក្រីស្តាល😁_ARK-SurvivalEvolved-QLd1...,Hello Guys\nHope you guys enjoy this video\nDo...,QLd1nGvjks4
3,#17អា១នេះខ្លាំងណាស់😁_ARK-SurvivalEvolved-AppGu...,Hello Guys\nHope you guys enjoy this video\nDo...,AppGuqwRp-4
4,#18បានលំពែងហើយតែ..😁_ARK-SurvivalEvolved-bdqzFE...,Hello Guys\nHope you guys enjoy this video\nDo...,bdqzFEg-pDE
...,...,...,...
7453,🙌BOOKREVIEW🙌January-oZ_qiYc133U.en.csv,"Before this video begins, I'm doing another gi...",oZ_qiYc133U
7454,🚙LIVE-LançamentoMITSUBISHIPAJEROSPORT2021-Blog...,Hello good afternoon\neveryone is very welcome...,VoLmhn8oEt4
7455,🛵ONTESTEunKIT70!🛵UnevéritableFUSEE!⚡+codePROMO...,all good ready to go to 70 that's it it's the ...,OdVpbg_O-a8
7456,🤡best&worstbooksi'vereadAKAaverylatemidyearfre...,That's why I ended\nup rating it 3 stars. It's...,phbkN8-4bBg


In [266]:
videos_df.to_excel(path_bases + 'subtitles.xlsx')

In [267]:
videos_df.to_csv(path_bases + 'subtitles.csv')

In [268]:
import joblib
joblib.dump(videos_df, path_bases + 'subtitles.pkl')

['C:\\Users\\Pichau\\Desktop\\PROJETO 3 - PLN\\bases_coletadas\\subtitles.pkl']